In [ ]:
# default_exp nbeats.callbacks

In [ ]:
# hide
import sys

sys.path.append("..")
import pandas as pd
%load_ext autoreload
%autoreload 2

/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
/home/tako/dev/env37/lib/python3.7/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


# N-Beats Callbacks

> A basic architecture for time series forecasting.


The approach is based on https://arxiv.org/abs/1905.10437


In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
from fastcore.utils import *
from fastcore.imports import *
from fastai2.basics import *
from fastai2.callback.hook import num_features_model
from fastai2.callback.all import *
from fastai2.torch_core import *
from torch.autograd import Variable
from fastseq.all import *

from fastseq.nbeats.model import *

## Metrics 

In [ ]:
#export
class NBeatsLossPart(Metric):
    "The loss according to the `loss_func` on a particular part of the time-serie."
    def __init__(self, start, end, name, *args, **kwargs):
        store_attr(self,"start,end")
        self._name = name
        
    def reset(self):           self.total,self.count = 0.,0
    def accumulate(self, learn):
        bs = find_bs(learn.yb)      
        pred, truth = learn.pred, learn.yb[0]
        if len(pred.shape) == 2:
            pred = pred[:,None,:]
            truth = truth[:,None,:]
        assert pred[:,0,self.start:self.end].shape == truth[:,0,self.start:self.end].shape 
        loss = to_detach(learn.loss_func(pred[:,0,self.start:self.end], truth[:,0,self.start:self.end])) / truth[:,0,self.start:self.end].shape[-1]
        self.total += loss.mean()*bs        
        self.count += bs
    @property
    def value(self): return self.total/self.count if self.count != 0 else None
    @property
    def name(self):  return self._name

In [ ]:
# hide
horizon, lookback = 10,40
items = dummy_data_generator(75, 10, nrows=10)
data = TSDataLoaders.from_items(items, horizon = horizon, lookback=lookback, step=3, valid_pct=.5
                               )
data = TSDataLoaders.from_items(items, horizon = horizon,lookback = lookback, bs=32)
mdl = NBeatsNet(device = data.train.device, horizon=horizon, lookback=lookback)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, metrics=[NBeatsLossPart(0,-10,'Last')],
#                 cbs=L(NBeatsTrainer())
               )
learn.loss_func
learn.fit(2,.1)

(1, 85)
Train:40; Valid: 20; Test 10
(1, 85)
Train:10; Valid: 160; Test 10


epoch,train_loss,valid_loss,Last,time
0,1.335197,3.763645,0.078127,00:00
1,5.021630,4.713816,0.101483,00:00


In [ ]:
# hide
path = untar_data(URLs.m4_daily)
data = TSDataLoaders.from_folder(path, horizon = horizon, lookback = lookback, nrows = 3, step=3, max_std=5)
mdl = NBeatsNet(device = data.train.device, horizon=horizon, lookback=lookback)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, metrics=[NBeatsLossPart(0,-10,'Last')],
#                 cbs=L(NBeatsTrainer())
               )
learn.loss_func
learn.fit(2,.1)

torch.Size([1, 1020])
Train:644; Valid: 18; Test 3


epoch,train_loss,valid_loss,Last,time
0,3.777536,2.382557,0.040267,00:01
1,2.936579,1.758298,0.027263,00:01


In [ ]:
#export
class NBeatsBackwards(NBeatsLossPart):
    "The loss according to the `loss_func` on the backwards part of the time-serie."    
    def __init__(self, lookback, *args, **kwargs):
        super().__init__(0, lookback, 'b_loss', *args, **kwargs)

In [ ]:
# hide
items = dummy_data_generator(50, 10, nrows=3)
data = TSDataLoaders.from_items(items, horizon = horizon,lookback = lookback, bs=32)
mdl = NBeatsNet(device = data.train.device, horizon=horizon, lookback=lookback)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, metrics=[NBeatsBackwards(lookback)],
#                 cbs=L(NBeatsTrainer())
               )
learn.loss_func
learn.fit(2,.1)

(1, 60)
Need to pad 3/3 time series due to length.
Train:3; Valid: 3; Test 3


epoch,train_loss,valid_loss,b_loss,time
0,1.112305,4.181420,0.069730,00:00
1,2.660353,6.155954,0.097354,00:00


In [ ]:
#export
class NBeatsForward(NBeatsLossPart):
    "The loss according to the `loss_func` on the forward part of the time-serie."  
    def __init__(self, lookback, *args, **kwargs):
        super().__init__(lookback, None, 'f_loss', *args, **kwargs)

In [ ]:
# hide
items = dummy_data_generator(50, 10, nrows=3)
data = TSDataLoaders.from_items(items, horizon = horizon,lookback = lookback, bs=32)
mdl = NBeatsNet(device = data.train.device, horizon=horizon, lookback=lookback)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, metrics=[NBeatsForward(lookback)],
               )
learn.loss_func
learn.fit(3,.1)
test_eq(type(learn.metrics[0].value), Tensor)

(1, 60)
Need to pad 3/3 time series due to length.
Train:3; Valid: 3; Test 3


epoch,train_loss,valid_loss,f_loss,time
0,1.235949,7.458382,0.358423,00:00
1,3.915194,5.401644,0.281416,00:00
2,5.551182,11.299735,0.405245,00:00


In [ ]:
path = untar_data(URLs.m4_daily)
data = TSDataLoaders.from_folder(path, horizon = horizon, lookback = lookback, nrows = 3, step=3, max_std=5)
mdl = NBeatsNet(device = data.train.device, horizon=horizon, lookback=lookback)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, metrics=L(None)+L(mae, smape, NBeatsTheta(), 
                                         NBeatsBackwards(lookback), NBeatsForward(lookback)),)

learn.fit(3, .1)

torch.Size([1, 1020])
Train:644; Valid: 18; Test 3


NameError: name 'NBeatsTheta' is not defined

In [ ]:
learn.show_results()

In [ ]:
# export
def _get_key_from_nested_dct(dct, s_key, exclude = [], namespace=''):
    r = {}
    for key in dct.keys():
        if sum([exc in key for exc in exclude])== 0 :
            if type(dct[key]) == dict:
                r.update(_get_key_from_nested_dct(dct[key], s_key, exclude, namespace=namespace+key))
            if s_key in key:
                r[namespace+key] = dct[key]
    return r

In [ ]:
dct = {'foo':{'bar':1},'bar':2,'foo2':{'foo3':3},'ignore':{'bar':1000}}
r = _get_key_from_nested_dct(dct,'bar',['ignore'])
test_eq(r,{'foobar': 1, 'bar': 2})

In [ ]:
#export
class NBeatsTheta(Metric):
    "The sqaure of the `theta` for every block. "
    def reset(self):           self.total,self.count = 0.,0
    def accumulate(self, learn):
        bs = find_bs(learn.yb)   
        theta_dct = _get_key_from_nested_dct(learn.model.dct,'theta',['bias','total','att'])
        t = torch.sum(tensor([v.float().abs().mean() for k,v in theta_dct.items()]))
        self.total += to_detach(t.abs().mean())*bs
        self.count += bs
    @property
    def value(self): return self.total/self.count if self.count != 0 else None
    @property
    def name(self):  return "theta"

In [ ]:
horizon, lookback = 7,10
path = untar_data(URLs.m4_daily)
data = TSDataLoaders.from_folder(path, horizon = horizon, lookback = lookback, nrows = 3, step=3, max_std=5)

mdl = NBeatsNet(device = data.train.device, horizon=horizon, lookback=lookback)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, metrics=[NBeatsTheta()],
               )

learn.fit(3,.1)
test_eq(type(learn.metrics[0].value),Tensor)

## Callbacks

In [ ]:
# export
class NBeatsAttention(Callback):  
    def means(self, df=True):
        theta_means = {k.replace('theta',''):v.float().cpu().data for k,v in _get_key_from_nested_dct(self.learn.model.dct,'theta',['total']).items()}
        ret = {}
        for k,v in theta_means.items():
            ret[k] = {}
            for i in range(v.shape[-1]):
                ret[k].update({'theta_'+str(i)+'_mean': v[:,i].mean().numpy(),
                               'theta_'+str(i)+'_std': v[:,i].std().numpy(),
                              })
            
        att = {k.replace('attention','att_mean'):v.float().cpu().numpy() for k,v in _get_key_from_nested_dct(self.learn.model.dct,'att',['total']).items()}
        for k in ret.keys():
            for att_key in att.keys():
                if k in att_key:
                    ret[k].update({'att_mean':att[att_key].mean(),
                                   'att_std':att[att_key].std(),
                                  })
                
        if df:
            return pd.DataFrame(ret)
        return ret

In [ ]:
horizon, lookback = 7,10
items = L(np.arange(-5,100)[None,:],np.arange(500,550)[None,:],np.arange(-110,-56)[None,:]).map(tensor)
data = TSDataLoaders.from_items(items, horizon = horizon, lookback=lookback, step=1, after_batch = NormalizeTS()
                             )
mdl = NBeatsNet(device = data.train.device,horizon=horizon, lookback=lookback)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, metrics=L(mae, smape, NBeatsTheta(), 
                                         NBeatsBackwards(lookback), NBeatsForward(lookback)),
                cbs=L( NBeatsAttention()
                     )
               )
learn.fit(3,.1)
df = learn.n_beats_attention.means()
df

In [ ]:
learn.show_results()

In [ ]:
# export
def CombinedLoss(loss_func, lookback, ratio = [1,1]):
    def _inner(pred, truth, *args, **kwargs):
        if len(pred.shape) == 2:
            pred = pred[:,None,:]
            truth = truth[:,None,:]
        loss = loss_func(pred[:,:,:lookback],truth[:,:,:lookback], *args, **kwargs) * ratio[0]
        loss += loss_func(pred[:,:,lookback:],truth[:,:,lookback:], *args, **kwargs) * ratio[1]
        return loss
    
    return _inner
  

In [ ]:
lookback = 7
y, y_hat = torch.arange(10).float()[None,None,:], torch.arange(10)[None,None,:].float()+torch.randn(10)
loss_fnc = CombinedLoss(F.mse_loss,lookback)
test_eq(F.mse_loss(y[:,:,:7],y_hat[:,:,:7])+F.mse_loss(y[:,:,7:],y_hat[:,:,7:]),loss_fnc(y, y_hat))

r = 10
loss_fnc = CombinedLoss(F.mse_loss, lookback, ratio = [1,r])
loss = loss_fnc(y, y_hat)
test_eq(F.mse_loss(y[:,:,:7],y_hat[:,:,:7])+F.mse_loss(y[:,:,7:],y_hat[:,:,7:])*r,loss)

In [ ]:
# hide
y, y_hat = torch.arange(10).float()[None,:], torch.arange(10)[None,:].float()+torch.randn(10)
loss_fnc = CombinedLoss(F.mse_loss,lookback)
test_eq(F.mse_loss(y[:,:7],y_hat[:,:7])+F.mse_loss(y[:,7:],y_hat[:,7:]),loss_fnc(y, y_hat))

In [ ]:
# export
class ClipLoss(Callback):
    "`Callback` that adds weights regularization the thetas in N-Beats training."
    def __init__(self, clip=5):
        self.clip = tensor([clip])

    def after_loss(self):
        self.learn.loss = torch.clamp(self.learn.loss, 0, self.clip.numpy()[0])
        
#     def after_backward(self):
#         nn.utils.clip_grad_norm_(self.learn.model.parameters(), self.clip)

In [ ]:
horizon, lookback = 2,10
items = L(np.arange(-5,30)[None,:],np.arange(50)[None,:]).map(tensor)
items[-1][:,-8:-5] = 1e10
data = TSDataLoaders.from_items(items, horizon = horizon, lookback=lookback, step=1, after_batch = NormalizeTS()
                             )
mdl = NBeatsNet(device = data.train.device,horizon=horizon, lookback=lookback)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, 
                cbs=L(ClipLoss()
                     )
               )

learn.fit(10,.1)
learn.recorder.plot_loss()

In [ ]:
learn.show_results()

In [ ]:
# hide
# old stuff
###################################################
###################################################

In [ ]:
# hide
class NBeatsBLoss(Callback): 
    def __init__(self,alpha = 1):store_attr(self,'alpha')
    def after_loss(self):
#         print('pre',self.learn.loss,self.n_beats_trainer.out['total_b_loss'].mean())
        self.learn.loss = self.learn.loss + self.n_beats_trainer.out['total_b_loss'].mean()*self.alpha
#         print('after',self.learn.loss)

In [ ]:
# hide
horizon, lookback = 7,30
items = L(np.arange(-5,100)[None,:],np.arange(250,550)[None,:],np.arange(-110,-56)[None,:]).map(tensor)
data = TSDataLoaders.from_items(items, horizon = horizon, lookback=lookback, step=1, after_batch = NormalizeTS()
                               )
thetas_dim= (2,4)
mdl = NBeatsNet(device = data.train.device, nb_blocks_per_stack = 1, horizon=horizon, lookback=lookback, thetas_dim=thetas_dim)
loss_func = F.mse_loss
learn = Learner(data, mdl, loss_func=loss_func, opt_func= Adam, metrics = [smape,NBeatsBackwards(lookback)],
                cbs=L(NBeatsAttention(),
                     )
               )

In [ ]:
# hide
# test_eq(list(df.columns),[o+str(i)+'_0' for i,o in enumerate(stack_types)])
# test_eq('att_mean' in list(df.axes[0]), True)
# test_eq('att_std' in list(df.axes[0]), True)

In [ ]:
# hide
# TODO maybe add extra backwards loss also in a callback??
class NBeatsTrainer(Callback):
    "`Callback` that adds weights regularization the thetas in N-Beats training."
    def __init__(self, theta=0., b_loss=0.): 
        self.theta, self.b_loss = theta, b_loss
        self.metrics = {'theta':tensor([0.]), 'b_loss':tensor([0.])}
        self.b = None

    def begin_train(self): 
        self.out = defaultdict(dict)
        self.metrics = {'theta':tensor([0.]), 'b_loss':tensor([0.])}
        
    def begin_validate(self): 
        self.out = defaultdict(dict)
        self.metrics = {'theta':tensor([0.]), 'b_loss':tensor([0.])}
        
    def after_pred(self):
        self.b = self.pred[1] 
        self.pred[2]['total_b'] = self.pred[1] 
        self.out = concat_dct(self.pred[2], self.out)   
        self.learn.pred = self.pred[0]

    def after_loss(self):        
        # theta
        value=tensor([0.])
        for key in self.out.keys():
            if 'bias' not in key and 'total' not in key and 'att' not in key:
                v = self.out[key]['theta'].float().pow(2).mean()
                if self.theta != 0.:     
                    self.learn.loss += self.theta * v.item()
                value = value + v
        self.metrics['theta'] += value.clone().cpu().detach()
        
        # backwards 
        value = self.learn.loss_func(self.b.float(), *self.xb, reduction='mean') 
        if self.b_loss != 0.:
            self.learn.loss += self.b_loss * value.mean() 
        self.metrics['b_loss'] += value.sum().clone().detach()
            


In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.external.ipynb.
Converted 02_data.load.ipynb.
Converted 03_data.core.ipynb.
Converted 05_nbeats.models.ipynb.
Converted 06_nbeats.callbacks.ipynb.
Converted 07_nbeats.learner.ipynb.
Converted 08_nbeats.interpret.ipynb.
Converted 11_metrics.ipynb.
Converted 12_compare.ipynb.
Converted index.ipynb.
